In [7]:
import os
import cv2
import numpy as np
import scipy.io as sio
from tqdm import tqdm

### Dataset folders

In [8]:
IMG_DIR = "data/ShanghaiTech/part_A/train_data/images"
GT_DIR  = "data/ShanghaiTech/part_A/train_data/ground-truth"
OUTPUT_IMG_DIR = "processed_images"
OUTPUT_GT_DIR  = "processed_points"

os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)
os.makedirs(OUTPUT_GT_DIR, exist_ok=True)

### Resize images and scale points

In [9]:
TARGET_SIZE = (512, 512)  # width, height

def scale_points(points, orig_w, orig_h, new_w, new_h):
    scaled = []
    scale_x = new_w / orig_w
    scale_y = new_h / orig_h
    
    for x, y in points:
        scaled.append([x * scale_x, y * scale_y])
    return np.array(scaled)

for img_name in tqdm(os.listdir(IMG_DIR)):
    if img_name.endswith(".jpg"):
        
        img_path = os.path.join(IMG_DIR, img_name)
        img = cv2.imread(img_path)
        orig_h, orig_w = img.shape[:2]

        resized_img = cv2.resize(img, TARGET_SIZE)
        
        mat_path = os.path.join(GT_DIR, "GT_" + img_name.replace(".jpg", ".mat"))
        mat = sio.loadmat(mat_path)
        points = mat["image_info"][0][0][0][0][0]

        points_scaled = scale_points(points, orig_w, orig_h, TARGET_SIZE[0], TARGET_SIZE[1])

        save_img_path = os.path.join(OUTPUT_IMG_DIR, img_name)
        cv2.imwrite(save_img_path, resized_img)

        save_points_path = os.path.join(OUTPUT_GT_DIR, img_name.replace(".jpg", ".npy"))
        np.save(save_points_path, points_scaled)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:15<00:00, 19.53it/s]


### Density Map Generation

In [10]:
from scipy.ndimage import gaussian_filter

In [11]:
IMG_DIR = "processed_images"      
POINTS_DIR = "processed_points"     
OUTPUT_DENSITY_DIR = "density_maps"

os.makedirs(OUTPUT_DENSITY_DIR, exist_ok=True)

In [12]:
def generate_density_map(points, h, w, sigma=4):
    density = np.zeros((h, w), dtype=np.float32)
    
    for x, y in points:
        if 0 <= int(x) < w and 0 <= int(y) < h:
            density[int(y), int(x)] = 1
    
    density = gaussian_filter(density, sigma=sigma)
    return density

In [13]:
for img_name in tqdm(os.listdir(IMG_DIR)):
    if img_name.endswith(".jpg"):
        
        img_path = os.path.join(IMG_DIR, img_name)
        img = cv2.imread(img_path)
        h, w = img.shape[:2]

        points_path = os.path.join(POINTS_DIR, img_name.replace(".jpg", ".npy"))
        points = np.load(points_path)

        density_map = generate_density_map(points, h, w, sigma=4)

        np.save(os.path.join(OUTPUT_DENSITY_DIR, img_name.replace(".jpg", ".npy")), density_map)

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:31<00:00,  9.65it/s]
